In [ ]:
%pip install python-docx

# 문서 읽기

In [ ]:
from docx import Document

document = Document('./tax.docx')
print(f'document == {dir(document)}')

full_text = ''

for index, paragraph in enumerate(document.paragraphs):
    print(f'paragraph = {paragraph.text}')
    full_text += f'{paragraph.text}\n'

# 문서 쪼개기

In [ ]:
%pip install tiktoken

In [ ]:
import tiktoken

def split_text(full_text, chunk_size):
    encoder = tiktoken.encoding_for_model("gpt-4o")
    total_encoding = encoder.encode(full_text)
    total_token_count = len(total_encoding)
    text_list = []
    
    for i in range(0, total_token_count, chunk_size):
        chunk = total_encoding[i: i+chunk_size]
        decoded = encoder.decode(chunk)
        text_list.append(decoded)
        
    return text_list

In [ ]:
chunk_list = split_text(full_text, 1500)

In [ ]:
%pip install chromadb

In [ ]:
import chromadb

chroma_client = chromadb.Client()

# 임베딩

In [ ]:
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()

openai_embedding = OpenAIEmbeddingFunction(
    model_name='text-embedding-3-large'
)

In [ ]:
collection_name = 'tax_collection_3'
tax_collection = chroma_client.create_collection(collection_name, embedding_function=openai_embedding)

In [ ]:
id_list = []
for index in range(len(chunk_list)):
    id_list.append(f'{index}')

In [ ]:
tax_collection.add(
    documents=chunk_list,
    ids=id_list
)

# 유사도 검색

In [ ]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrived_docs = tax_collection.query(query_texts=query, n_results=3)

In [18]:
retrived_docs['documents'][0]

['. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31., 2025. 10. 1.>\n1. 「공익신탁법」에 따른 공익신탁의 이익\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n다. 대통령령으로 정하는 농어가부업소득\n라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.\n바. 대통령령으로 정하는 작물재배업에서 발생하는 소득\n사. 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n3. 근로소득과 퇴직소득 중 다음 각 목의 어느 하나에 해당하는 소득\n가. 대통령령으로 정하는 복무 중인 병(兵)이 받는 급여\n나. 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n다. 「산업재해보상보험법」에 따라 수급권자가 받는 요양급여, 휴업급여, 장해급여, 간병

# LLM 질의

In [ ]:
%pip install openai

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": f"당신은 한국의 소득세 전문가입니다, 아래 내용을 참고해서 사용자의 질문에 답변해주세요. {retrived_docs['documents'][0]}"},
        {"role": "user", "content": query}
    ]
)

In [17]:
response.choices[0].message.content

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 몇 가지 공제를 먼저 고려해야 합니다. 다음은 일반적인 공제 및 세율을 기준으로한 예시입니다.\n\n1. **근로소득공제**: 근로소득공제는 연봉 금액에 따라 계산됩니다.\n   - 연봉 5천만원의 경우, 근로소득공제는 대략 다음과 같이 계산할 수 있습니다:\n     - 5천만원 이하 구간: 연 550만원 + (총급여 – 5천만원) × 30%\n\n2. **과세표준**: 총급여액에서 근로소득공제를 빼면 과세표준이 나옵니다.\n   - 과세표준 = 총급여 - 근로소득공제\n   - 이 과세표준 금액에서 기본공제 및 기타 다양한 공제를 차감해야 하나, 이를 일반적인 소득공제로 대신할 수 있습니다.\n\n3. **세율 적용**: 2023년 기준 소득세율은 다음과 같으며, 과세표준에 따라 적용됩니다.\n   - 1,200만원 이하: 6%\n   - 1,200만원 초과 ~ 4,600만원 이하: 15%\n   - 4,600만원 초과 ~ 8,800만원 이하: 24%\n   - 8,800만원 초과 ~ 1억5천만원 이하: 35%\n   - 1억5천만원 초과 ~ 3억원 이하: 38%\n   - 3억원 초과 ~ 5억원 이하: 40%\n   - 5억원 초과: 42%\n\n4. **세액 계산**: 과세표준에 맞는 해당 구간의 세율을 적용하여 세액을 계산합니다.\n\n5. **세액공제**: 산출된 세액에서 세액공제를 통해 최종 소득세를 산출합니다.\n\n위의 과정은 기본적인 소득세 계산의 예시이며, 좀 더 정확한 계산을 위해 개인의 세금 공제 항목, 의료비, 교육비, 기부금 공제 등 다른 많은 요소들을 고려해야 합니다. 따라서 구체적인 소득세 계산을 위해서는 국세청의 연말정산 프로그램이나 세무 전문가의 도움을 받는 것이 좋습니다.'